In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
Train=pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv')
Test= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv')
sample_submission= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [2]:
#Filter BY CO2 emissions (metric tons per capita)
specific_string = 'CO2 emissions (metric tons per capita)'
filtered_rows = Train[Train['Indicator'] == specific_string]
filtered_rows

,Country Code,Country Name,Indicator,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
11,Afg,Afghanistan,CO2 emissions (metric tons per capita),0.055166607,0.055292719,0.066810138,0.073004968,0.054866759,0.080361134,0.089756909,0.109940042,0.165573355,0.239506901,0.304230216,0.408965291,0.335061035,0.298087813,0.283692396,0.297971775
23,Alb,Albania,CO2 emissions (metric tons per capita),1.031567545,1.056868354,1.233001531,1.361158778,1.427944204,1.360092207,1.363988602,1.433796507,1.428249586,1.492116704,1.64276193,1.768108509,1.565921402,1.656389503,1.795712442,1.665218525
35,Alg,Algeria,CO2 emissions (metric tons per capita),2.601065339,2.520609526,2.605694613,2.751098761,2.752758782,2.85792657,2.985149131,3.023502915,3.116770947,3.187069044,3.184357,3.305232517,3.621367718,3.658468679,3.811521147,3.951960939
47,Ame,American Samoa,CO2 emissions (metric tons per capita),..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
59,And,Andorra,CO2 emissions (metric tons per capita),7.927016355,7.725626659,7.498764979,7.238069466,7.286756008,7.206273645,6.805400082,6.890389929,7.081822365,6.995396198,7.223590934,6.957586407,6.86229282,6.674233189,6.445930663,6.485769241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3143,Sub,Sub-Saharan Africa,CO2 emissions (metric tons per capita),0.742866043,0.787827764,0.770312004,0.793663742,0.816417132,0.799086697,0.776663194,0.776076215,0.80414679,0.755561429,0.786758499,0.763961095,0.771660846,0.790531658,0.802872437,0.76398389
3155,Sub,Sub-Saharan Africa (excluding high income),CO2 emissions (metric tons per capita),0.742518227,0.787450847,0.769905273,0.793282981,0.815993983,0.798675052,0.776252936,0.77565827,0.80372919,0.755114664,0.786334412,0.763583523,0.771278211,0.790174495,0.802481827,0.763559116
3167,Sub,Sub-Saharan Africa (IDA & IBRD countries),CO2 emissions (metric tons per capita),0.742866043,0.787827764,0.770312004,0.793663742,0.816417132,0.799086697,0.776663194,0.776076215,0.80414679,0.755561429,0.786758499,0.763961095,0.771660846,0.790531658,0.802872437,0.76398389
3179,Upp,Upper middle income,CO2 emissions (metric tons per capita),3.167140886,3.243556127,3.347641763,3.646680369,3.955406019,4.246087066,4.524939703,4.770829501,4.86797795,4.944131035,5.315663058,5.673918008,5.764481643,5.885587659,5.857718499,5.735859454


In [3]:
#Drop Indicator Column
filtered_rows.drop(columns=['Indicator'], inplace=True)
filtered_rows

/tmp/ipykernel_18/1368651258.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows.drop(columns=['Indicator'], inplace=True)


,Country Code,Country Name,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
11,Afg,Afghanistan,0.055166607,0.055292719,0.066810138,0.073004968,0.054866759,0.080361134,0.089756909,0.109940042,0.165573355,0.239506901,0.304230216,0.408965291,0.335061035,0.298087813,0.283692396,0.297971775
23,Alb,Albania,1.031567545,1.056868354,1.233001531,1.361158778,1.427944204,1.360092207,1.363988602,1.433796507,1.428249586,1.492116704,1.64276193,1.768108509,1.565921402,1.656389503,1.795712442,1.665218525
35,Alg,Algeria,2.601065339,2.520609526,2.605694613,2.751098761,2.752758782,2.85792657,2.985149131,3.023502915,3.116770947,3.187069044,3.184357,3.305232517,3.621367718,3.658468679,3.811521147,3.951960939
47,Ame,American Samoa,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
59,And,Andorra,7.927016355,7.725626659,7.498764979,7.238069466,7.286756008,7.206273645,6.805400082,6.890389929,7.081822365,6.995396198,7.223590934,6.957586407,6.86229282,6.674233189,6.445930663,6.485769241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3143,Sub,Sub-Saharan Africa,0.742866043,0.787827764,0.770312004,0.793663742,0.816417132,0.799086697,0.776663194,0.776076215,0.80414679,0.755561429,0.786758499,0.763961095,0.771660846,0.790531658,0.802872437,0.76398389
3155,Sub,Sub-Saharan Africa (excluding high income),0.742518227,0.787450847,0.769905273,0.793282981,0.815993983,0.798675052,0.776252936,0.77565827,0.80372919,0.755114664,0.786334412,0.763583523,0.771278211,0.790174495,0.802481827,0.763559116
3167,Sub,Sub-Saharan Africa (IDA & IBRD countries),0.742866043,0.787827764,0.770312004,0.793663742,0.816417132,0.799086697,0.776663194,0.776076215,0.80414679,0.755561429,0.786758499,0.763961095,0.771660846,0.790531658,0.802872437,0.76398389
3179,Upp,Upper middle income,3.167140886,3.243556127,3.347641763,3.646680369,3.955406019,4.246087066,4.524939703,4.770829501,4.86797795,4.944131035,5.315663058,5.673918008,5.764481643,5.885587659,5.857718499,5.735859454


In [4]:
#Drop Country Code Column
filtered_rows.drop(columns=['Country Code'], inplace=True)
filtered_rows

/tmp/ipykernel_18/3319185666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows.drop(columns=['Country Code'], inplace=True)


,Country Name,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
11,Afghanistan,0.055166607,0.055292719,0.066810138,0.073004968,0.054866759,0.080361134,0.089756909,0.109940042,0.165573355,0.239506901,0.304230216,0.408965291,0.335061035,0.298087813,0.283692396,0.297971775
23,Albania,1.031567545,1.056868354,1.233001531,1.361158778,1.427944204,1.360092207,1.363988602,1.433796507,1.428249586,1.492116704,1.64276193,1.768108509,1.565921402,1.656389503,1.795712442,1.665218525
35,Algeria,2.601065339,2.520609526,2.605694613,2.751098761,2.752758782,2.85792657,2.985149131,3.023502915,3.116770947,3.187069044,3.184357,3.305232517,3.621367718,3.658468679,3.811521147,3.951960939
47,American Samoa,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
59,Andorra,7.927016355,7.725626659,7.498764979,7.238069466,7.286756008,7.206273645,6.805400082,6.890389929,7.081822365,6.995396198,7.223590934,6.957586407,6.86229282,6.674233189,6.445930663,6.485769241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3143,Sub-Saharan Africa,0.742866043,0.787827764,0.770312004,0.793663742,0.816417132,0.799086697,0.776663194,0.776076215,0.80414679,0.755561429,0.786758499,0.763961095,0.771660846,0.790531658,0.802872437,0.76398389
3155,Sub-Saharan Africa (excluding high income),0.742518227,0.787450847,0.769905273,0.793282981,0.815993983,0.798675052,0.776252936,0.77565827,0.80372919,0.755114664,0.786334412,0.763583523,0.771278211,0.790174495,0.802481827,0.763559116
3167,Sub-Saharan Africa (IDA & IBRD countries),0.742866043,0.787827764,0.770312004,0.793663742,0.816417132,0.799086697,0.776663194,0.776076215,0.80414679,0.755561429,0.786758499,0.763961095,0.771660846,0.790531658,0.802872437,0.76398389
3179,Upper middle income,3.167140886,3.243556127,3.347641763,3.646680369,3.955406019,4.246087066,4.524939703,4.770829501,4.86797795,4.944131035,5.315663058,5.673918008,5.764481643,5.885587659,5.857718499,5.735859454


In [5]:
#Covert Year Column in rows
melted_df = filtered_rows.melt(id_vars='Country Name', var_name='Year', value_name='CO2_Emissions')
melted_df

,Country Name,Year,CO2_Emissions
0,Afghanistan,2000 [YR2000],0.055166607
1,Albania,2000 [YR2000],1.031567545
2,Algeria,2000 [YR2000],2.601065339
3,American Samoa,2000 [YR2000],..
4,Andorra,2000 [YR2000],7.927016355
...,...,...,...
4251,Sub-Saharan Africa,2015 [YR2015],0.76398389
4252,Sub-Saharan Africa (excluding high income),2015 [YR2015],0.763559116
4253,Sub-Saharan Africa (IDA & IBRD countries),2015 [YR2015],0.76398389
4254,Upper middle income,2015 [YR2015],5.735859454


In [6]:
#Sort Row in accending order by Year and Country Name
sorted_df = melted_df.groupby('Country Name').apply(lambda x: x.sort_values('Year')).reset_index(drop=True)
sorted_df.head(20)

/tmp/ipykernel_18/1831615012.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = melted_df.groupby('Country Name').apply(lambda x: x.sort_values('Year')).reset_index(drop=True)


,Country Name,Year,CO2_Emissions
0,Afghanistan,2000 [YR2000],0.055166607
1,Afghanistan,2001 [YR2001],0.055292719
2,Afghanistan,2002 [YR2002],0.066810138
3,Afghanistan,2003 [YR2003],0.073004968
4,Afghanistan,2004 [YR2004],0.054866759
5,Afghanistan,2005 [YR2005],0.080361134
6,Afghanistan,2006 [YR2006],0.089756909
7,Afghanistan,2007 [YR2007],0.109940042
8,Afghanistan,2008 [YR2008],0.165573355
9,Afghanistan,2009 [YR2009],0.239506901


In [7]:
#All country name into unique_values
unique_values = melted_df['Country Name'].unique()
unique_values.shape

(266,)

In [8]:
new_value = 0.00
sorted_df = sorted_df.replace('..', new_value)

In [9]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")
warnings.resetwarnings()
total=0
j=0
for i in unique_values:
    afg = sorted_df[sorted_df['Country Name'] == i]
    # Define SARIMAX parameters
    order = (1, 1, 0)  # (p, d, q)
    seasonal_order = (1, 1, 0, 6)  # (P, D, Q, s)
    # Fit SARIMAX model
    model = SARIMAX(afg['CO2_Emissions'].astype(float), order=order, seasonal_order=seasonal_order)
    result = model.fit()

    # Predictions
    forecast_steps = 16  # Number of steps to forecast
    forecast = result.forecast(steps=forecast_steps)
    row_index = sample_submission[sample_submission.eq(i).any(axis=1)].index[0]


    new_values = [i, forecast[16], forecast[17],forecast[18], forecast[19], forecast[20],forecast[30]]
    sample_submission.loc[row_index] = new_values



/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:997: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'
 This problem is unconstrained.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -8.47049D-01    |proj g|=  1.05029D+00

At iterate    5    f= -8.50732D-01    |proj g|=  1.68780D-01

At iterate   10    f= -9.12263D-01    |proj g|=  2.54783D-01

At iterate   15    f= -9.25596D-01    |proj g|=  1.13664D+00

At iterate   20    f= -9.71111D-01    |proj g|=  2.94077D-01

At iterate   25    f= -9.72204D-01    |proj g|=  4.62155D-03
  ys=-1.387E-05  -gs= 1.230E-05 BFGS update SKIPPED

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Proj

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/conda/lib/python3.10/site-pa


At iterate   10    f= -2.86945D-02    |proj g|=  8.62698D-04

At iterate   15    f= -2.86984D-02    |proj g|=  1.37060D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     18     27      1     0     0   5.802D-05  -2.870D-02
  F =  -2.8698389923090824E-002

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.92176D-01    |proj g|=  6.63267D-01

At iterate    5    f= -6.93306D-01    |proj g|=  2.60533D-02

At iter

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given w

...


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.
/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index wi


At iterate   15    f= -1.21385D+00    |proj g|=  3.39641D-03
  ys=-2.355E-10  -gs= 2.400E-10 BFGS update SKIPPED

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     19    128      3     1     0   6.944D-03  -1.214D+00
  F =  -1.2138451731471049     

ABNORMAL_TERMINATION_IN_LNSRCH                              
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -1.21145D+00    |proj g|=  6.32185D-01

At iterate    5    f= -1.21161D+00    |proj g|=  1.42311D-01



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.
/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index wi


At iterate   10    f= -1.21402D+00    |proj g|=  3.20904D-01

At iterate   15    f= -1.21405D+00    |proj g|=  4.43962D-03

At iterate   20    f= -1.21407D+00    |proj g|=  1.50989D-02

At iterate   25    f= -1.21407D+00    |proj g|=  1.46711D-01
  ys=-1.673E-06  -gs= 4.958E-07 BFGS update SKIPPED

At iterate   30    f= -1.21408D+00    |proj g|=  1.18004D-02
  ys=-2.484E-09  -gs= 1.512E-08 BFGS update SKIPPED

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     32    120      2     2     0   7.909D-03  -1.214D+00
  F =  -1.2140805749906491     

ABNORMAL_TERMINATION_IN_LNSRCH                              
RUNNING T


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.
/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index wi

In [10]:
sample_submission.head(5)

,Country Name,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2030 [YR2030]
0,Afghanistan,0.287907,0.320518,0.323758,0.339657,0.389872,0.556750
1,Albania,1.866266,1.935703,1.777523,1.862128,1.975866,2.194618
2,Algeria,3.957503,4.069786,4.266931,4.304901,4.420214,5.550121
3,American Samoa,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Andorra,6.568557,6.451696,6.110751,6.142191,6.251308,5.385008


In [11]:
sample_submission.to_csv('submission.csv', index=False)